https://jira.x5.ru/browse/CVMXC-2797

In [1]:
name = 'CVMXC-2797_rusagro_pa_add'

In [5]:
!pip install pydantic
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

Looking in indexes: https://repo-dmp.do.x5.ru/repository/pypi/simple


In [6]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/09 15:44:48 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [7]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

In [8]:
data = pd.read_excel('camp_data.xlsx')

### 1

In [116]:
i = 0

In [117]:
camp_id = data.iloc[i, 0]

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        data.iloc[i, 7]: "sms_viber_tg",
        data.iloc[i, 9]: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': str(datetime.datetime.date(data.iloc[i,4])),
    'date_communication_to': str(datetime.datetime.date(data.iloc[i,5])),

    'wave_ids': [
        data.iloc[i, 8] #если нет, то -1
    ],
    
    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [118]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [119]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [120]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [121]:
plu_codes = list(map(lambda x: int(x), data.iloc[i,2].split(',')))

In [122]:
seg_pd_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg, type_data = 'with_rto')

In [123]:
seg_pd_cg['camp_name'] = camp_id 
seg_pd_cg['camp_num'] = i + 1

In [124]:
val = seg_pd_cg.copy()

### 2

In [125]:
i = 1

In [126]:
camp_id = data.iloc[i, 0]

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        data.iloc[i, 7]: "sms_viber_tg",
        data.iloc[i, 9]: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': str(datetime.datetime.date(data.iloc[i,4])),
    'date_communication_to': str(datetime.datetime.date(data.iloc[i,5])),

    'wave_ids': [
        data.iloc[i, 8] #если нет, то -1
    ],
    
    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [127]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [128]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [129]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [130]:
plu_codes = list(map(lambda x: int(x), data.iloc[i,2].split(',')))

In [131]:
seg_pd_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg, type_data = 'with_rto')

In [132]:
seg_pd_cg['camp_name'] = camp_id 
seg_pd_cg['camp_num'] = i + 1

In [133]:
val = val.append(seg_pd_cg)

In [134]:
val

,plu_code,plu_nm,plu_qty,rto_novat,rto_vat,camp_name,camp_num
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,117.000,14613.12,17535.86,X-30-5-Rusagro-Moscow-Heavy-sof,1
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,353.000,24053.63,28864.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
2,3495835,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл,18.000,1361.11,1633.34,X-30-5-Rusagro-Moscow-Heavy-sof,1
3,3195297,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл,112.000,6073.34,7287.83,X-30-5-Rusagro-Moscow-Heavy-sof,1
4,3982578,МОС.ПРОВАНС.Майонез оливковый 67% 600мл,9.000,1080.29,1296.31,X-30-5-Rusagro-Moscow-Heavy-sof,1
5,3444610,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл,14.000,874.59,1049.53,X-30-5-Rusagro-Moscow-Heavy-sof,1
6,4102644,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл,15.000,572.68,687.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
7,3695167,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл,41.000,4058.91,4870.74,X-30-5-Rusagro-Moscow-Heavy-sof,1
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,401.000,48877.04,58653.00,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,740.000,56220.57,67463.52,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2


### 3

In [135]:
i = 2

In [136]:
camp_id = data.iloc[i, 0]

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        data.iloc[i, 7]: "sms_viber_tg",
        data.iloc[i, 9]: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': str(datetime.datetime.date(data.iloc[i,4])),
    'date_communication_to': str(datetime.datetime.date(data.iloc[i,5])),

    'wave_ids': [
        data.iloc[i, 8] #если нет, то -1
    ],
    
    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [137]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [138]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [139]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [140]:
plu_codes = list(map(lambda x: int(x), data.iloc[i,2].split(',')))

In [141]:
seg_pd_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg, type_data = 'with_rto')

In [142]:
seg_pd_cg['camp_name'] = camp_id 
seg_pd_cg['camp_num'] = i + 1

In [143]:
val = val.append(seg_pd_cg)

In [144]:
val

,plu_code,plu_nm,plu_qty,rto_novat,rto_vat,camp_name,camp_num
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,117.000,14613.12,17535.86,X-30-5-Rusagro-Moscow-Heavy-sof,1
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,353.000,24053.63,28864.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
2,3495835,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл,18.000,1361.11,1633.34,X-30-5-Rusagro-Moscow-Heavy-sof,1
3,3195297,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл,112.000,6073.34,7287.83,X-30-5-Rusagro-Moscow-Heavy-sof,1
4,3982578,МОС.ПРОВАНС.Майонез оливковый 67% 600мл,9.000,1080.29,1296.31,X-30-5-Rusagro-Moscow-Heavy-sof,1
5,3444610,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл,14.000,874.59,1049.53,X-30-5-Rusagro-Moscow-Heavy-sof,1
6,4102644,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл,15.000,572.68,687.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
7,3695167,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл,41.000,4058.91,4870.74,X-30-5-Rusagro-Moscow-Heavy-sof,1
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,401.000,48877.04,58653.00,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,740.000,56220.57,67463.52,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2


### 4

In [145]:
i = 3

In [146]:
camp_id = data.iloc[i, 0]

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        data.iloc[i, 7]: "sms_viber_tg",
        data.iloc[i, 9]: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': str(datetime.datetime.date(data.iloc[i,4])),
    'date_communication_to': str(datetime.datetime.date(data.iloc[i,5])),

    'wave_ids': [
        data.iloc[i, 8] #если нет, то -1
    ],
    
    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [147]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [148]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [149]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [150]:
plu_codes = list(map(lambda x: int(x), data.iloc[i,2].split(',')))

In [151]:
seg_pd_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg, type_data = 'with_rto')

[Stage 183:=====================================================> (38 + 1) / 39]22/03/09 21:03:57 ERROR client.TransportResponseHandler: Still have 2 requests outstanding when connection from /192.168.234.57:53530 is closed


In [152]:
seg_pd_cg['camp_name'] = camp_id 
seg_pd_cg['camp_num'] = i + 1

In [153]:
val = val.append(seg_pd_cg)

In [154]:
val

,plu_code,plu_nm,plu_qty,rto_novat,rto_vat,camp_name,camp_num
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,117.000,14613.12,17535.86,X-30-5-Rusagro-Moscow-Heavy-sof,1
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,353.000,24053.63,28864.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
2,3495835,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл,18.000,1361.11,1633.34,X-30-5-Rusagro-Moscow-Heavy-sof,1
3,3195297,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл,112.000,6073.34,7287.83,X-30-5-Rusagro-Moscow-Heavy-sof,1
4,3982578,МОС.ПРОВАНС.Майонез оливковый 67% 600мл,9.000,1080.29,1296.31,X-30-5-Rusagro-Moscow-Heavy-sof,1
5,3444610,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл,14.000,874.59,1049.53,X-30-5-Rusagro-Moscow-Heavy-sof,1
6,4102644,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл,15.000,572.68,687.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
7,3695167,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл,41.000,4058.91,4870.74,X-30-5-Rusagro-Moscow-Heavy-sof,1
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,401.000,48877.04,58653.00,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,740.000,56220.57,67463.52,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2


### 5

In [155]:
i = 4

In [156]:
camp_id = data.iloc[i, 0]

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        data.iloc[i, 7]: "sms_viber_tg",
        data.iloc[i, 9]: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': str(datetime.datetime.date(data.iloc[i,4])),
    'date_communication_to': str(datetime.datetime.date(data.iloc[i,5])),

    'wave_ids': [
        data.iloc[i, 8] #если нет, то -1
    ],
    
    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [157]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [158]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [159]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [160]:
plu_codes = list(map(lambda x: int(x), data.iloc[i,2].split(',')))

In [161]:
seg_pd_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg, type_data = 'with_rto')

In [162]:
seg_pd_cg['camp_name'] = camp_id 
seg_pd_cg['camp_num'] = i + 1

In [163]:
val = val.append(seg_pd_cg)

In [164]:
val

,plu_code,plu_nm,plu_qty,rto_novat,rto_vat,camp_name,camp_num
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,117.000,14613.12,17535.86,X-30-5-Rusagro-Moscow-Heavy-sof,1
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,353.000,24053.63,28864.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
2,3495835,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл,18.000,1361.11,1633.34,X-30-5-Rusagro-Moscow-Heavy-sof,1
3,3195297,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл,112.000,6073.34,7287.83,X-30-5-Rusagro-Moscow-Heavy-sof,1
4,3982578,МОС.ПРОВАНС.Майонез оливковый 67% 600мл,9.000,1080.29,1296.31,X-30-5-Rusagro-Moscow-Heavy-sof,1
5,3444610,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл,14.000,874.59,1049.53,X-30-5-Rusagro-Moscow-Heavy-sof,1
6,4102644,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл,15.000,572.68,687.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
7,3695167,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл,41.000,4058.91,4870.74,X-30-5-Rusagro-Moscow-Heavy-sof,1
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,401.000,48877.04,58653.00,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,740.000,56220.57,67463.52,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2


### 6

In [165]:
i = 5

In [166]:
camp_id = data.iloc[i, 0]

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        data.iloc[i, 7]: "sms_viber_tg",
        data.iloc[i, 9]: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': str(datetime.datetime.date(data.iloc[i,4])),
    'date_communication_to': str(datetime.datetime.date(data.iloc[i,5])),

    'wave_ids': [
        data.iloc[i, 8] #если нет, то -1
    ],
    
    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [167]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [168]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [169]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [170]:
plu_codes = list(map(lambda x: int(x), data.iloc[i,2].split(',')))

In [171]:
seg_pd_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg, type_data = 'with_rto')

In [172]:
seg_pd_cg['camp_name'] = camp_id 
seg_pd_cg['camp_num'] = i + 1

In [173]:
val = val.append(seg_pd_cg)

In [174]:
val

,plu_code,plu_nm,plu_qty,rto_novat,rto_vat,camp_name,camp_num
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,117.000,14613.12,17535.86,X-30-5-Rusagro-Moscow-Heavy-sof,1
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,353.000,24053.63,28864.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
2,3495835,МОС.ПРОВАНС.Майонез ДОМАШ.55% д/п 390мл,18.000,1361.11,1633.34,X-30-5-Rusagro-Moscow-Heavy-sof,1
3,3195297,МОС.ПРОВАНС.Майонез КЛАСС.67% 220мл,112.000,6073.34,7287.83,X-30-5-Rusagro-Moscow-Heavy-sof,1
4,3982578,МОС.ПРОВАНС.Майонез оливковый 67% 600мл,9.000,1080.29,1296.31,X-30-5-Rusagro-Moscow-Heavy-sof,1
5,3444610,МОС.ПРОВАНС.Майонез СЛИВОЧНЫЙ 67% 390мл,14.000,874.59,1049.53,X-30-5-Rusagro-Moscow-Heavy-sof,1
6,4102644,МОС.ПРОВАНС.Соус ЛЕГКИЙ май.30%д/п 200мл,15.000,572.68,687.14,X-30-5-Rusagro-Moscow-Heavy-sof,1
7,3695167,МОС.ПРОВАНС.Майон.на переп.яйц.67% 600мл,41.000,4058.91,4870.74,X-30-5-Rusagro-Moscow-Heavy-sof,1
0,4112954,МОС.ПРОВАНС.Майонез КЛАССИЧ.67%д/п 700мл,401.000,48877.04,58653.00,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2
1,4105769,Я ЛЮБ.ГОТ.Майон.МОСК.ПРОВ.КЛ.67%д/п390мл,740.000,56220.57,67463.52,U-29-russagro-mosk.provansal-postavschik-1- Heavy,2


### 7

In [175]:
i = 6

In [176]:
camp_id = data.iloc[i, 0]

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        data.iloc[i, 7]: "sms_viber_tg",
        data.iloc[i, 9]: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': str(datetime.datetime.date(data.iloc[i,4])),
    'date_communication_to': str(datetime.datetime.date(data.iloc[i,5])),

    'wave_ids': [
        data.iloc[i, 8] #если нет, то -1
    ],
    
    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [177]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [178]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [179]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [180]:
plu_codes = list(map(lambda x: int(x), data.iloc[i,2].split(',')))

In [181]:
seg_pd_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg, type_data = 'with_rto')

In [182]:
seg_pd_cg['camp_name'] = camp_id 
seg_pd_cg['camp_num'] = i + 1

In [183]:
val = val.append(seg_pd_cg)

In [185]:
val = val.reset_index(drop=True)

In [186]:
val.to_excel('camp_info.xlsx', index=False)

22/03/12 10:02:07 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state KILLED! Check the YARN application logs for more details.
22/03/12 10:02:07 ERROR cluster.YarnClientSchedulerBackend: Diagnostics message: Application killed by user.
22/03/12 10:02:07 ERROR client.TransportClient: Failed to send RPC RPC 8163751796626175860 to /192.168.234.57:44032: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/03/12 10:02:07 ERROR cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Sending RequestExecutors(0,0,Map(),Set()) to AM was unsuccessful
java.io.IOException: Failed to send RPC RPC 8163751796626175860 to /192.168.234.57:44032: java.nio.channels.ClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.java:362)
	at org.apache.spark.network.client.TransportClient$StdChann